# 1. Librerías & Set Up

👉🏻 Instalaremos todas las librerias necesarias para poder trabajar con PySpark y Neo4j

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f9afdc26155c116c36fb07b48f4cfdcda3a6663f4f240660b11b087768ca9ed7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 2.4 MB/s eta 0:00:00


In [3]:
from pyspark.sql import SparkSession
from itertools import product

In [4]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [5]:
# Iniciamos una sesion en PySpark
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Neo4j Graphs and others examples

👉🏻 Las siguientes funciones permiten cargar grafos de ejemplos para poder realizar las consultas de los patrones de grafo.

👉🏻 Existe la funcion create_neo4j_graph para poder cargar una base de datos en Neo4j y luego obtener las aristas existentes de este grafo.

👉🏻 Existe la funcion load_graph para poder seleccionar el grafo que queremos realizar las consultas.

In [6]:
def load_neo4j_graph():
  """
  Funcion para cargar la base de datos de twitter como ejemplo de grafo en Neo4j.
  """
  # Realizamos la conexion con la base de datos del grafo guardado en Neo4j.
  NEO4J_URI = "neo4j+s://demo.neo4jlabs.com"
  NEO4J_USERNAME = "twitter"
  NEO4J_PASSWORD = "twitter"
  AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

  driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
  with driver.session() as session:
      try:
          session.run("RETURN 1")
          print("Connection to Neo4j established successfully!")
      except Exception as e:
          print(f"Failed to connect to Neo4j: {e}")
      # Query que permite cargar todas las aristas de cada usuario
      # (descartamos el usuario Neo4j para tener una cantidad mas variada xd,
      # ya que si lo dejabamos se repetia mucho)
      query = """
      MATCH (u1:User)-[r:INTERACTS_WITH | SIMILAR_TO]->(u2:User)
      WHERE u1.name <> "Neo4j" AND u2.name <> "Neo4j"
      RETURN u1.name as user1, type(r) AS relationship, u2.name AS user2
      """
      result = session.run(query)
      df_edges = pd.DataFrame(result.data())
      relationship = df_edges['relationship'].unique()

      # Definimos una clase para cada relacion ...
      relationship_class = {i : rel for rel, i in enumerate(relationship)}
      for inter, inter_class in relationship_class.items():
          print(f"{inter} AS class {inter_class}")
      df_edges['relationship'].replace(relationship_class, inplace = True)

      # Transformamos cada usuario a un numero (su identificador) en base
      # a su nombre de usuari que debe ser unico ...
      users1 = df_edges['user1'].unique().tolist()
      users2 = df_edges['user2'].unique().tolist()
      users = list(set(users1 + users2))
      users_id = {user : id for id, user in enumerate(users)}
      df_edges['user1'].replace(users_id, inplace = True)
      df_edges['user2'].replace(users_id, inplace = True)

      np.save('users_id', users_id, allow_pickle = True)
      np.save('relationship_class', relationship_class, allow_pickle = True)

      # Cargamos el grafo en una lista de tuplas
      neo4f_graph = [(item[0], item[1], item[2]) for item in df_edges.values]
  driver.close()
  return neo4f_graph


In [7]:
def load_graph(option):
  """
  Funcion que selecciona el grafo de ejemplo, retorna una lista de tuplas
  de las aristas del grafo.
  """
  if option == "neo4j-graph":
    neo4f_graph = load_neo4j_graph()
    return neo4f_graph

  elif option == "basic-example":
    return [
        (1,11,2), (1,11,3), (2,11,3), (3,11,2),
        (3,11,4), (4,11,1), (4,11,2), (4,11,3),
        (4,12,5), (5,12,1), (5,12,2), (5,12,6),
        (1,12,2), (2,12,3), (3,12,1)
    ]
  else:
    print("La opcion no existe :(")
    return None

# 3. MapReduce Algorithm Implementation

👉🏻 Las siguientes funciones permiten desarrollar el algoritmo MapReduce implementado con PySpark

In [8]:
def hash(n):
  """
  Returns number mod 2. The ouput will be 0 or 1.
  """
  return n % 2

👉🏻 Las funciones siguientes permiten simular la fase de mapeo del algoritmo.

In [9]:
def get_keys(edge, b_dim, b_set, pattern_dim):
  """
  output: retorna las llaves correspondientes para un vertice.

  Idea general: buscamos el par hash_n1,hash_n2 dentro de las posibles
  combinaciones dentro del espacio de imagenes de la funcion de hash.
  Dentro del for, obtenemos un string con la codificacion de las llaves y luego
  verificamos si es una llave candidata para el vertice entregado:

  1.  sequence_in_reducer: Si la secuencia 'b1b2' esta en la llave del reducer
      codificada como hash(n1)hash(n2) ?, donde ? = 0 o 1, entonces se considerará
      el par reducer_key : edge.
  2.  edge_case: El otro caso, es para cuando tenemos por ejemplo x = n2 y z = n1
      para patrones de 3 vertices.
  """

  hash_n1 = hash(edge[0]) # valor de hash para el nodo 1 = b1
  hash_n2 = hash(edge[2]) # valor de hash para el nodo 2 = b2
  values = [] # posible keys
  sequence = '{}{}'.format(hash_n1, hash_n2)

  for i in range(0, b_dim ** pattern_dim):
    reducer = ''.join(str(num) for num in b_set[i])
    sequence_in_reducer = sequence in reducer
    edge_case = reducer[0] == sequence[1] and reducer[pattern_dim - 1] == sequence[0]
    if sequence_in_reducer or edge_case:
        reducer_key = tuple(int(digit) for digit in reducer)
        values.append((reducer_key, edge))

  return values


def map_phase(rdd, b_dim, b_set, pattern_dim):
  """
  input:
    - rdd: RDD del grafo de dimension 'dim'
    - b_dim: Cantidad de elementos de las imagenes de la funcion de hash.
    - b_set: Imagenes de la funcion de hash.
    - pattern_dim: cantidad de nodos del patron de grafo.
  ouput: Mapeo de cada arista con respecto a las llaves
  """

  mapped_keys = rdd.flatMap(lambda edge: get_keys(edge, b_dim, b_set, pattern_dim))

  return mapped_keys

👉🏻 Las siguientes funciones permiten simular la fase de reduce del algoritmo.

In [10]:
def find_patterns(edges, pattern_dim):
    """
    Funcion que retorna todos los patrones encontrados de la forma
    (n1,...,nl) : [[(n1,label,n2), ..., (nl-1,label,nl)], ... ]
    donde (n1,...,nl) corresponde a los nodos que forman el patron
    y la llave corresponde a los patrones posibles formados por estos nodos
    para una etique
    """
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim)
    return cycles


def get_neighbors(edges):
    """
    Funcion que retorna todas las aristas consecutivas a otras aristas.
    Es decir, aristas vecinas.
    """
    neighbors = []
    for i in range(len(edges)):
        curr_edge = edges[i]
        neighbors.append([])
        for j in range(len(edges)):
            next_edge = edges[j]
            if next_edge == curr_edge:
                continue
            if curr_edge[2] == next_edge[0]:
                neighbors[i].append(next_edge)
    return neighbors


def find_cycles(edges, neighbors, pattern_dim):
    """
    Funcion que verifica si existe algun grafo ciclico en el grafo de una
    dimension pattern_dim. Retorna todos los ciclos encontrados.
    Utilizamos el algoritmo DFS iterativo para la busqueda de ciclos.
    """
    cycles = [] # Lista de ciclos encontrados

    for edge in edges: # Por cada arista del grafo
        visited = set() # Determinamos las aritas que ya han sido visitadas
        # Inicializamos el stack con con la
        stack = [(edge, [edge])]
        # arista donde empezamos el recorrido

        while len(stack):
            curr_edge, path = stack.pop() # Extraemos la primera arista del stack
            # Recorremos esta arista, por lo que la marcamos como visitada
            visited.add(curr_edge)
            # Caso 1: Si el camino recorrido actual supera la cantidad de nodos del bgp,
            #         descartamos este camino
            if len(path) > pattern_dim:
                continue
            # Caso 2: Si el camino recorrido tiene un largo igual a pattern_dim,
            #         y son aristas transitivas, entonces un posible ciclo.
            first_node = path[0][0] # primer nodo del camino
            last_node = path[-1][2] # ultimo nodo del caminmo
            if len(path) == pattern_dim and first_node == last_node:
                cycles.append(path)
                # Agregamos el camino y seguimos recorriendo
                continue
            # Si no es ciclico el camino desde el nodo actual,
            # entonces iteramos dentro de sus vecinos
            for neighbor_edge in neighbors[edges.index(curr_edge)]:
                if neighbor_edge not in visited:
                    stack.append((neighbor_edge, path + [neighbor_edge]))

    return cycles


def get_nodes(pattern, pattern_dim):
    """
    Funcion que retorna la tupla con los nodos que forman un ciclo.
    """
    nodes = []
    for i in range(pattern_dim):
      edge = pattern[i]
      nodes.append(edge[0])

    return (tuple(sorted(nodes)), tuple(sorted(pattern)))


def get_unique_lists(pattern1, pattern2):
    """
    Funcion que elimina patrones de grafo duplicados para cada llave
    (n1,n2, ..., nl). Si tenemos l = 3, entonces seria (n1,n2,n3)
    """
    lst = []
    if pattern2 != pattern1:
      return lst.append(pattern2)
    else:
      return pattern1


def reduce_phase(mapped_keys, pattern_dim):
    """
    input: RDD del grafo y cantidad de nodos del patron de grafo.
    output: patrones encontrados.
    """
    # Agrupamos por llaves
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Filtramos solamente los grupos conformados por 3 aristas ...
    reducers_edges = reducers.map(lambda v: v[1]) \
                    .filter(lambda x: len(x) >= pattern_dim)
    # Por cada grupo buscamos un patron de grafo
    patterns = reducers_edges \
              .map(lambda edges: find_patterns(edges, pattern_dim)) \
              .flatMap(list) \
              .map(lambda pattern: get_nodes(pattern, pattern_dim)) \
              .groupByKey().mapValues(lambda v: list(set(v)))


    return patterns

👉🏻 Funcion final que simula el algoritmo MapReduce. Recibe los parametros correspondientes del grafo; las dimensiones del conjunto de imagenes de la funcion de hash; el conjunto de imagenes de la funcion de hash; la dimension del patron de grafo.

In [11]:
def map_reduce(rdd_graph, b_dim, b_set, pattern_dim, show=False):
  """
  Funcion que simula el algoritmo MapReduce, en donde se distribuye el grafo
  en diferentes reducers y luego obtenemos los posibles patrones
  de grafos formados combinando las informacion de todos los reducers.
  """
  # Fase de Map: Obtenemos las llaves de cada reducer y el conjunto de aristas mapeados a estas llaves.
  mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)

  if show: # Mostras las llaves mapeadas
    print("Fase Map: \n")
    for key in mapped_keys.collect():
      print(key)
    print("\nFase Reduce\n")

  # Fase Reduce: Obtenemos todos los posibles patrones de L nodos.
  patterns = reduce_phase(mapped_keys, pattern_dim)
  return patterns

# 3. Aplicacion MapReduce en Ejemplo Basico

👉🏻 En esta seccion aplicaremos el algoritmo MapReduce para un grafo de prueba pequeño (el que fue dado al inicio de la tarea, con algunas modificaciones). Este lo guardamos como 'basic-example'.

In [184]:
graph = load_graph('basic-example')
rdd_graph = sc.parallelize(graph)
rdd_graph.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6),
 (1, 12, 2),
 (2, 12, 3),
 (3, 12, 1)]

## 3.1 MapReduce Algorithm for Triangles

👉🏻 Para encontrar triangulos dentro de un grafo utilizaremos el algoritmo Map Reduce para parametros b_0 = 2 y l = 3.

👉🏻 En este caso, b_0=2 ya que al utilizar la funcion de hash modular `f(x) = x mod 2`, generamos un conjunto de 2 imagenes: {0,1}. En consecuencia, los posibles reducers son:
```markdown
REDUCERS = [(0, 0, 0),
            (0, 0, 1),
            (0, 1, 0),
            (0, 1, 1),
            (1, 0, 0),
            (1, 0, 1),
            (1, 1, 0),
            (1, 1, 1)]
```

👉🏻 Dado que estamos buscando subgrafos de 3 nodos, entonces l_0 = 3.

👉🏻 Las aristas se distribuyen dentro de los reducers, por lo que almacenamos el grafo de manera distribuida.En cada reducer verificamos si las aristas generan triangulos, en el caso que si (formaria un ciclo) se almacenará en una lista y retornaremos los nodos correspondientes.




In [185]:
# Dimension de elementos del conjunto de imagenes de la funcion de hash: |{0,1}|
b_0 = 2
# Dimension del patron de grafo (triangulo para este caso)
l_0 = 3
# Conjunto de imagenes de la funcion de hash
b_set_0 = list(range(b_0))
# Posibles reducers
reducers_0 = list(product(b_set_0, repeat=l_0))

patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)

patterns.collect()

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

👉🏻 Al aplicar el algoritmo mapReduce, podemos notar que existen 3 tripletas de nodos que forman un triangulo. En donde estos nodos permiten formar diferentes patrones de grafos triangulares. En este caso, como tenemos mas de una etiqueta distinta, podemos notar que para una tripleta de nodo, forman mas de un triangulo.
```markdown
  (2,3,4); (1,2,3); (1,3,4)
```

👉🏻 Las siguientes funciones se definiran con el fin de desarrollar consultas sobre estos grafos.


In [186]:
def filter_patterns(pattern, pattern_dim, labels):
  """
  Funcion que retorna los patrones filtrados segun el label proporcionado como
  parametro.
  """
  pattern_labels = [edge[1] for edge in pattern]
  for i in range(pattern_dim + 1):
      # verificamos si coinciden los labels ...
      if pattern_labels == labels:
        return pattern
      # Hacemos un desplazamiento hacia la derecha ...
      pattern_labels = pattern_labels[i:] + pattern_labels[:i]

  return None

def bgp_query(rdd_graph, b_dim, reducers, pattern_dim, query):
  """
  Funcion que retorna todos los patrones de grafos de dimension "pattern_dim"
  que coincidan segun la query entregada.
  """

  patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)
  processed_query = query.replace("(", "").replace(")", "").split(",")
  labels = [int(part.strip()) for part in processed_query if part.strip().isdigit()]

  result = patterns.filter(lambda pattern: filter_patterns(pattern, pattern_dim, labels))
  return result

In [187]:
# Realizamos la siguiente consulta en donde queremos obtener todos los triangulos
# donde la etiqueta es 11, independiente del nodo.
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,11,y), (y,11,z), (z,11,w)")
for res in result.collect():
  print(res)

((2, 11, 3), (3, 11, 4), (4, 11, 2))
((1, 11, 3), (3, 11, 4), (4, 11, 1))


## 3.2 MapReduce Algorithm for Squares


👉🏻 Ahora aplicaremos nuestro algoritmo para la busqueda de patrones de grafos de 4 variables. En primer lugar, utilizaremos el mismo algoritmo anterior, pero cambiaremos los parametros de l y b_set. En este caso, utilizaremos l = 4. En consecuencia, obtenemos una cantidad de $2^4$ reducers (definidos como `reducers_1`).

In [188]:
# Como buscaremos los posibles cuadrados a formar, entonces tendremos l=4 nodos
# en el bgp.
l_1 = 4
# Tendremos b^4 reducers posibles.
b_set_1 = list(range(b_0))
reducers_1 = list(product(b_set_1, repeat=l_1))
# aplicamos el algoritmo para patrones de grafos de l=4 nodos, con 2^4 reducers
# y con un conjunto de preimagenes de la funcion de hash {0,1}.
squares_patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
squares_patterns.collect() # Obtenemos todos los posibles patrones cuadrados sin importar el label.

[((1, 2, 3, 4),
  [((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1))]),
 ((2, 3, 4, 5),
  [((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2))]),
 ((2, 3, 3, 4),
  [((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3)),
   ((2, 12, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))]),
 ((1, 3, 4, 5), [((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))]),
 ((1, 2, 3, 3), [((1, 11, 3), (2, 12, 3), (3, 11, 2), (3, 12, 1))])]

👉🏻 Ahora podemos notar que el algoritmo se demora aproximadamente 1s en correr.

👉🏻 Podemos notar que existen 5 tuplas que forman patrones de grafos cuadrilateros (considerando que tenemos diferentes etiquetas, por lo que algunos nodos se repiten, sin embargo se debe a que presentan labels de aristas distintos).

```markdown
(1,2,3,4), (2,3,4,5), (2,3,3,4), (1,2,4,5), y (1,2,3,3).
```

In [189]:
def filter_patterns_squares(pattern, pattern_dim, M1):
    """
    Funcion que retorna los patrones filtrados para cuadrilateros
    """
    for i in range(pattern_dim + 1):
        shift_patterns = pattern[i:] + pattern[:i]
        A = [item[0] for item in shift_patterns]  # Variables (A)
        L = list(set(item[1] for item in shift_patterns)) # Labels (L)

        dict_var_dir = {var: dir for dir, var in enumerate(A)}
        dict_label_dir = {label: dir for dir, label in enumerate(L)}
        # definimos la matriz AxLxA. Para que se realice match, la
        # matriz del patron de grafo debe ser la misma que la matriz
        # para el subgrafo
        M2 = np.zeros((len(A), len(L), len(A)))

        for item in shift_patterns:
            node_1 = item[0]
            label = int(item[1])
            node_2 = item[2]

            dir_node_1 = dict_var_dir[node_1]
            dir_label = dict_label_dir[label]
            dir_node_2 = dict_var_dir[node_2]

            M2[dir_node_1][dir_label][dir_node_2] = 1
        # si la matriz M1 (que es la de consulta) es igual a la matriz M2 (que
        # es la que se procesa para cada subgrafo), entonces se retorna el patron
        if np.array_equal(M1, M2):
            return pattern

    return None

def get_matches(rdd_graph, b_dim, reducers, pattern_dim, query):
    """
    Funcion que procesa la consulta y retorna los posibles matches segun 
    el patron de grafo que se encuentra. Idealmente, se debe entregara la 
    consulta como "[(x,label,y), (y,label,z), (z,label,w), (w,label,x)]"
    con las separaciones por comas entre tuplas. 
    """
    processed_query = query.replace(" ", ";").replace("(", "").replace(")", "").split(",;")
    processed_query = [item.split(",") for item in processed_query]

    A = [item[0] for item in processed_query]  # lista de Variables
    L = list(set(int(item[1]) for item in processed_query))  # lista de Labels

    # Diccionario en donde recordamos la direccion de memoria de cada variable
    # y label.
    dict_var_dir = {var: dir for dir, var in enumerate(A)}
    dict_label_dir = {label: dir for dir, label in enumerate(L)}

    # definimos nuestra matriz M de dimension A X L X A
    M1 = np.zeros((len(A), len(L), len(A)))
    for item in processed_query:
        node_1 = item[0]
        label = int(item[1])
        node_2 = item[2]
        dir_node_1 = dict_var_dir[node_1]
        dir_label = dict_label_dir[label]
        dir_node_2 = dict_var_dir[node_2]
        M1[dir_node_1][dir_label][dir_node_2] = 1

    # con map reduce, encontramos todos los patrones que tienen 4 aristas (cuadrilateros)
    patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)

    # filtramos los subgrafos segun el patron de grafo que se solicita
    result = patterns.filter(lambda pattern: filter_patterns_squares(pattern, pattern_dim, M1) is not None)
    return result.collect()


In [190]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,12,w), (w,12,x)")

[((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1)),
 ((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2)),
 ((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))]

In [191]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,11,w), (w,11,x)")

[((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1))]

👉🏻 Podemos notar que tambien podemos tener cuadrados pero considerando 3 nodos, pero existen 4 aristan en diferentes direcciones (Se veria visualmente como un "8"):
    
    3 -[11]-> 4 -[11]->3-[11]->2-[11]->3

In [192]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,x), (y,11,z), (z,11,y)")

[((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))]

# 4. Aplicacion de un grafo a gran escala: Twitter Graph

👉🏻 Ahora cargaremos un grafo de mayor tamaño (700 aristas) para evaluar nuestro algoritmo. El siguiente grafo corresponde a un grafo que modela la interaccion de cada usuario en la red social twitter.
El grafo sera cargado como lista de tuplas, en donde cada tupla modela la relacion entre un usuario y otro. Consideramos las siguientes 2 tipos de relaciones

    Relacion 0: INTERACTS_WITH - El usuario 1 interacciona con otro usuario 2
    Relacion 1: SIMILAR_TO - El usuario 1 es similar al usuario 2

Referencia: https://github.com/neo4j-graph-examples/twitter-v2


In [193]:
graph = load_graph('neo4j-graph')
rdd_graph = sc.parallelize(graph)

Connection to Neo4j established successfully!
INTERACTS_WITH AS class 0
SIMILAR_TO AS class 1


In [194]:
rdd_graph.count() # Cantidad de aristas del grafo

744

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Ahora aplicaremos MapReduce para encontrar los patrones de grafos con forma de triangulo dentro del grafo cargado desde Neo4j.



In [207]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)
patterns.collect()

[((23, 106, 396), [((23, 1, 106), (106, 1, 396), (396, 0, 23))]),
 ((23, 56, 396), [((23, 1, 396), (56, 1, 23), (396, 0, 56))]),
 ((106, 144, 175), [((106, 0, 144), (144, 0, 175), (175, 1, 106))]),
 ((23, 116, 408), [((23, 0, 408), (116, 1, 23), (408, 1, 116))]),
 ((23, 28, 186), [((23, 0, 186), (28, 1, 23), (186, 0, 28))]),
 ((23, 318, 396), [((23, 1, 396), (318, 1, 23), (396, 0, 318))]),
 ((23, 396, 408), [((23, 1, 396), (396, 0, 408), (408, 1, 23))]),
 ((285, 318, 396),
  [((285, 1, 318), (318, 1, 396), (396, 0, 285)),
   ((285, 1, 396), (318, 0, 285), (396, 0, 318))]),
 ((21, 374, 492),
  [((21, 1, 374), (374, 0, 492), (492, 1, 21)),
   ((21, 0, 492), (374, 0, 21), (492, 1, 374))]),
 ((52, 91, 118), [((52, 0, 118), (91, 1, 52), (118, 1, 91))]),
 ((144, 148, 155), [((144, 0, 155), (148, 1, 144), (155, 1, 148))]),
 ((134, 135, 144), [((134, 1, 144), (135, 1, 134), (144, 0, 135))]),
 ((144, 156, 323), [((144, 0, 323), (156, 1, 144), (323, 0, 156))]),
 ((106, 144, 231),
  [((106, 0, 14

👉🏻 El algoritmo es bien rapido de ejecutar, y podemos ver claramente las relaciones entre diferentes usuarios, para diferentes labels.

👉🏻 Para poder analizar las relaciones y compararlo con el grafo mostrado en Neo4j, realizaremos una consulta, y luego compararemos los resultados ...

In [208]:
# Cargamos el diccionario que guardamos los nombres de usuarios y el id asociado.
users_id = np.load('users_id.npy', allow_pickle=True).item()
dict_users_id = {value : key for key, value in users_id.items()}
# Cargamos el tipo de relacion.
relationship_class = np.load('relationship_class.npy', allow_pickle=True).item()
dict_relation_class = {value : key for key, value in relationship_class.items()}

In [211]:
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,1,y), (y,0,z), (z,0,x)")
result.collect()

[((23, 0, 186), (28, 1, 23), (186, 0, 28)),
 ((285, 1, 396), (318, 0, 285), (396, 0, 318)),
 ((144, 0, 323), (156, 1, 144), (323, 0, 156)),
 ((134, 0, 291), (148, 1, 134), (291, 0, 148)),
 ((135, 0, 236), (148, 1, 135), (236, 0, 148)),
 ((148, 0, 155), (155, 1, 236), (236, 0, 148)),
 ((135, 0, 291), (155, 1, 135), (291, 0, 155)),
 ((52, 0, 118), (118, 1, 348), (348, 0, 52)),
 ((134, 0, 236), (148, 1, 134), (236, 0, 148)),
 ((135, 1, 144), (144, 0, 231), (231, 0, 135)),
 ((135, 0, 148), (148, 1, 231), (231, 0, 135)),
 ((135, 0, 291), (148, 1, 135), (291, 0, 148)),
 ((155, 1, 236), (236, 0, 291), (291, 0, 155)),
 ((134, 0, 291), (155, 1, 134), (291, 0, 155))]

In [212]:
def load_usernames(pat):
  """
  Funcion que retorna el nombre de usuario codificado a un id, y tambien transforma
  la clase del label a su valor original
  """
  list_users = []
  for i in range(len(pat)):
    edge = pat[i]
    user1 = dict_users_id[edge[0]]
    label = dict_relation_class[edge[1]]
    user2 = dict_users_id[edge[2]]
    list_users.append((user1, label, user2))
  return list_users

👉🏻 Ahora con la consulta anterior, analizaremos los usuarios involucrados en el siguiente caso: el primer usuario interacciona con el segundo; el segundo es similar al tercero; y el tercero el similar al primero.

In [213]:
result.map(load_usernames).collect()

[[('Michael Hunger', 'INTERACTS_WITH', 'Ben'),
  ('Morgan | Tech Consultant | runtcpip.com 🔜 SXSW',
   'SIMILAR_TO',
   'Michael Hunger'),
  ('Ben', 'INTERACTS_WITH', 'Morgan | Tech Consultant | runtcpip.com 🔜 SXSW')],
 [('Nigel Small 🇪🇺', 'SIMILAR_TO', 'Michael Simons'),
  ('Jim Webber', 'INTERACTS_WITH', 'Nigel Small 🇪🇺'),
  ('Michael Simons', 'INTERACTS_WITH', 'Jim Webber')],
 [('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs', 'INTERACTS_WITH', 'Javelynn'),
  ('Françoise Morvan',
   'SIMILAR_TO',
   'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs'),
  ('Javelynn', 'INTERACTS_WITH', 'Françoise Morvan')],
 [('Born To Be Idle', 'INTERACTS_WITH', 'Brana Rakic'),
  ('Wen.S', 'SIMILAR_TO', 'Born To Be Idle'),
  ('Brana Rakic', 'INTERACTS_WITH', 'Wen.S')],
 [('John Wolpert', 'INTERACTS_WITH', 'Chill Zone'),
  ('Wen.S', 'SIMILAR_TO', 'John Wolpert'),
  ('Chill Zone', 'INTERACTS_WITH', 'Wen.S')],
 [('Wen.S', 'INTERACTS_WITH', 'guinnessstache'),
  ('guinnessstache', 'SIMILAR_TO', 'Chill Zone'),
  ('Chil

## 4.2 MapReduce For Squares

👉🏻 Ahora aplicaremos el algoritmo MapReduce para buscar patrones todos los posibles subgrafos de solo 4 variables, para luego realizar una consulta.

In [214]:
patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
patterns.collect()

[((116, 396, 408, 408),
  [((116, 0, 408), (396, 0, 408), (408, 1, 116), (408, 1, 396))]),
 ((56, 318, 396, 396),
  [((56, 1, 396), (318, 1, 396), (396, 0, 56), (396, 0, 318))]),
 ((56, 396, 396, 408),
  [((56, 1, 396), (396, 0, 56), (396, 0, 408), (408, 1, 396))]),
 ((318, 396, 396, 408),
  [((318, 1, 396), (396, 0, 318), (396, 0, 408), (408, 1, 396))]),
 ((52, 52, 118, 348),
  [((52, 0, 118), (52, 1, 348), (118, 1, 52), (348, 0, 52))]),
 ((314, 314, 328, 416),
  [((314, 0, 416), (314, 1, 328), (328, 0, 314), (416, 1, 314))]),
 ((314, 328, 416, 416),
  [((314, 0, 416), (328, 0, 416), (416, 1, 314), (416, 1, 328))]),
 ((50, 106, 144, 144),
  [((50, 1, 144), (106, 0, 144), (144, 0, 50), (144, 1, 106))]),
 ((134, 148, 236, 236),
  [((134, 0, 236), (148, 1, 236), (236, 0, 148), (236, 1, 134))]),
 ((134, 148, 148, 236),
  [((134, 0, 148), (148, 1, 134), (148, 1, 236), (236, 0, 148))]),
 ((134, 134, 148, 236),
  [((134, 0, 148), (134, 0, 236), (148, 1, 134), (236, 1, 134))]),
 ((314, 328, 3

👉🏻 Realizamos la siguiente consulta en donde obtener todos los usuarios tal que:
* El primero interacciona con el segundo
* El segundo interaccion con el tercero
* El tercero es similar al cuarto
* El cuarto es similar al primero

In [215]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,0,z), (z,1,w), (w,1,x)")

[((50, 1, 144), (144, 0, 175), (175, 0, 231), (231, 1, 50))]

👉🏻 Ahora realizaremos otras consultas para ver como andan los resultados :)


In [220]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,1,y), (y,0,z), (z,1,w), (w,1,x)")

[((23, 1, 106), (106, 0, 144), (144, 1, 275), (275, 1, 23)),
 ((23, 1, 106), (106, 1, 396), (396, 0, 408), (408, 1, 23))]

In [231]:
get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,1,z), (z,0,w), (w,0,x)")

[((106, 0, 144), (144, 0, 175), (175, 0, 231), (231, 1, 106)),
 ((134, 0, 135), (135, 0, 236), (236, 0, 291), (291, 1, 134)),
 ((148, 0, 155), (155, 1, 236), (236, 0, 291), (291, 0, 148)),
 ((144, 0, 175), (175, 0, 231), (231, 0, 323), (323, 1, 144))]

👉🏻 Ahora, para poder analizar quienes son los usuarios involucrados en la ultima consulta, mapearemos cada id con respecto a su username :)

In [232]:
for res in get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,1,z), (z,0,w), (w,0,x)"):
  print(load_usernames(res))

[('Lju Lazarevic', 'INTERACTS_WITH', 'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs'), ('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs', 'INTERACTS_WITH', 'Chidambara .ML.'), ('Chidambara .ML.', 'INTERACTS_WITH', 'Nancy Tremblay❤😀🌻🇨🇦'), ('Nancy Tremblay❤😀🌻🇨🇦', 'SIMILAR_TO', 'Lju Lazarevic')]
[('Born To Be Idle', 'INTERACTS_WITH', 'John Wolpert'), ('John Wolpert', 'INTERACTS_WITH', 'Chill Zone'), ('Chill Zone', 'INTERACTS_WITH', 'Brana Rakic'), ('Brana Rakic', 'SIMILAR_TO', 'Born To Be Idle')]
[('Wen.S', 'INTERACTS_WITH', 'guinnessstache'), ('guinnessstache', 'SIMILAR_TO', 'Chill Zone'), ('Chill Zone', 'INTERACTS_WITH', 'Brana Rakic'), ('Brana Rakic', 'INTERACTS_WITH', 'Wen.S')]
[('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs', 'INTERACTS_WITH', 'Chidambara .ML.'), ('Chidambara .ML.', 'INTERACTS_WITH', 'Nancy Tremblay❤😀🌻🇨🇦'), ('Nancy Tremblay❤😀🌻🇨🇦', 'INTERACTS_WITH', 'Javelynn'), ('Javelynn', 'SIMILAR_TO', 'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs')]


In [233]:
for res in get_matches(rdd_graph, b_0, reducers_1, l_1, "(x,1,y), (y,0,z), (z,1,w), (w,1,x)"):
  print(load_usernames(res))

[('Michael Hunger', 'SIMILAR_TO', 'Lju Lazarevic'), ('Lju Lazarevic', 'INTERACTS_WITH', 'Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs'), ('Dr.Salahideen ALHAJ 🔶 🔸️#WearAMask #SDGs', 'SIMILAR_TO', 'Taite'), ('Taite', 'SIMILAR_TO', 'Michael Hunger')]
[('Michael Hunger', 'SIMILAR_TO', 'Lju Lazarevic'), ('Lju Lazarevic', 'SIMILAR_TO', 'Michael Simons'), ('Michael Simons', 'INTERACTS_WITH', 'Lukas Eder'), ('Lukas Eder', 'SIMILAR_TO', 'Michael Hunger')]
